In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

def lee_archivo(archivo):
	f = open(archivo, "r")
	contenido = f.read()
	f.close()

	lines = contenido.split("\n")
	n = int(lines[0])
	points = [ list(map(float, lines[i].split("\t")))  for i in range(1, len(lines)-1 )]

	return n, points

In [ ]:
# Utilidad de dibujo (reutilizada con pequeño arreglo)
def draw_hull( puntos, elegidos, lowest, text):
	# Esto grafica todos los puntos con colores random, y semi-trasparentes
	colors = np.random.rand(len(puntos))
	plt.scatter([p[0] for p in puntos], [p[1] for p in puntos], c=colors, alpha=0.5)

	# Usa scatters similares para graficar los puntos elegidos y el lowest
	plt.scatter(lowest[0], lowest[1])

	plt.text(lowest[0], lowest[1], "lowest")


	# Usa plot para graficar el poligono
	elegidos.append(lowest)
	plt.plot([p[0] for p in elegidos], [p[1] for p in elegidos] )

	# text es el nombre de la instancia de prueba, agregado como texto a la grafica
	plt.title(text)

	plt.show()

In [ ]:
# Quickhull: utilidades de geometría
def _side(A, B, P):
	# >0 si P está a la izquierda de AB, <0 si a la derecha, 0 si colineal
	return (B[0]-A[0])*(P[1]-A[1]) - (B[1]-A[1])*(P[0]-A[0])

def _dist_line(A, B, P):
	# distancia perpendicular desde P a la recta AB (proporcional al área del triángulo)
	num = abs(_side(A, B, P))
	den = math.hypot(B[0]-A[0], B[1]-A[1])
	return num/den if den != 0 else 0.0

def _build_hull(P, Q, S):
	# S: subconjunto de puntos estrictamente a la izquierda de PQ
	if not S:
		return [P]
	# 1) Punto más alejado de PQ
	C = max(S, key=lambda X: _dist_line(P, Q, X))
	# 2) Particionar en los que quedan a la izquierda de PC y CQ
	S1 = [X for X in S if _side(P, C, X) > 0]
	S2 = [X for X in S if _side(C, Q, X) > 0]
	# 3) Recurse y concatenar
	return _build_hull(P, C, S1) + _build_hull(C, Q, S2)

In [ ]:
def find_hull_quickhull(n, puntos):
	# Casos pequeños: la envolvente es el propio conjunto ordenado
	if n <= 1:
		lowest_point = min(puntos, key=lambda x: x[1]) if n == 1 else [0.0, 0.0]
		return [puntos.copy(), lowest_point]
	if n == 2:
		lowest_point = min(puntos, key=lambda x: x[1])
		return [puntos.copy(), lowest_point]

	# 1) extremos por x para dividir en dos mitades
	A = min(puntos, key=lambda p: (p[0], p[1]))
	B = max(puntos, key=lambda p: (p[0], p[1]))

	left = [P for P in puntos if _side(A, B, P) > 0]
	right = [P for P in puntos if _side(A, B, P) < 0]

	# 2) construir las dos cadenas del casco
	left_chain = _build_hull(A, B, left)
	right_chain = _build_hull(B, A, right)
	hull = left_chain + right_chain + [B]

	# 3) eliminar duplicados preservando el orden
	seen = set()
	ordered = []
	for p in hull:
		key = (p[0], p[1])
		if key not in seen:
			ordered.append([p[0], p[1]])
			seen.add(key)

	# 4) rotar para iniciar en el mismo "lowest_point" que usa el notebook original
	lowest_point = min(puntos, key=lambda x: x[1])
	if lowest_point in ordered:
		idx = ordered.index(lowest_point)
		ordered = ordered[idx:] + ordered[:idx]

	return [ordered, lowest_point]

In [ ]:
# Por cada instancia....

instancias = ["puntos-n8.txt", "puntos-n10.txt",
             "puntos-n11.txt", "puntos-n15.txt",
             "puntos-n16.txt", "puntos-n20.txt",
             "puntos-n50.txt", "puntos-n100.txt"]

# 1. Lee los puntos


for file_name in instancias:
	n, puntos = lee_archivo( file_name )

	# 2. Calcula la convex hull (Quickhull)
	hull, lowest = find_hull_quickhull(n, puntos)

	# 3. Y grafica...
	draw_hull(puntos, hull, lowest, file_name)